# PySpark & PyDeequ Shakeout


In [1]:
%%bash
pip install pydeequ

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession, Row, DataFrame

import sagemaker_pyspark
import pydeequ

classpath = ":".join(sagemaker_pyspark.classpath_jars())

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

Please set env variable SPARK_VERSION


:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6d03278e-8b7d-4189-af76-866647cd6739;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1

22/11/23 20:04:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
%%bash

echo $JAVA_HOME
$JAVA_HOME/bin/java -version

/home/ec2-user/anaconda3/envs/JupyterSystemEnv


openjdk version "11.0.1" 2018-10-16 LTS
OpenJDK Runtime Environment Zulu11.2+3 (build 11.0.1+13-LTS)
OpenJDK 64-Bit Server VM Zulu11.2+3 (build 11.0.1+13-LTS, mixed mode)


In [4]:
# Read a CSV file from local instance directory into a Spark dataframe
file_input = 'data/TOP_FedClct_20221001_20221001.csv'

df = spark.read \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load(file_input) 

df.printSchema()
df.show(5)

# Write a copy of the Spark dataframe to a Parquet output file
file_output = file_input.replace('.csv', '.parquet')
df.write.mode('overwrite').parquet(file_output)
print(f'File output: {file_output}')


root
 |-- Record Date: timestamp (nullable = true)
 |-- Creditor Agency Name: string (nullable = true)
 |-- Agency ID: integer (nullable = true)
 |-- Agency Site Type Code: string (nullable = true)
 |-- Agency Site ID: string (nullable = true)
 |-- Agency Site Name: string (nullable = true)
 |-- Source Agency Alpha: string (nullable = true)
 |-- Source Agency Numeric: string (nullable = true)
 |-- Source Agency Site ID: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Payment Agency ID: string (nullable = true)
 |-- Agency Type Code: string (nullable = true)
 |-- Payment Source Description: string (nullable = true)
 |-- Payment Source Code: string (nullable = true)
 |-- Payment Category Description: string (nullable = true)
 |-- Payment Type Description: string (nullable = true)
 |-- Payment Type Code: string (nullable = true)
 |-- Payment Type ID: integer (nullable = true)
 |-- Debt Type Code: string (nullable = true)
 |-- Debt Type Description: string (nullabl

File output: data/TOP_FedClct_20221001_20221001.parquet


In [5]:
# Read a Parquet file from local instance directory
file_input = 'data/TOP_FedClct_20221001_20221001.parquet'

df = spark.read.parquet(file_input) 
df.printSchema()
df.show(5)


root
 |-- Record Date: timestamp (nullable = true)
 |-- Creditor Agency Name: string (nullable = true)
 |-- Agency ID: integer (nullable = true)
 |-- Agency Site Type Code: string (nullable = true)
 |-- Agency Site ID: string (nullable = true)
 |-- Agency Site Name: string (nullable = true)
 |-- Source Agency Alpha: string (nullable = true)
 |-- Source Agency Numeric: string (nullable = true)
 |-- Source Agency Site ID: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Payment Agency ID: string (nullable = true)
 |-- Agency Type Code: string (nullable = true)
 |-- Payment Source Description: string (nullable = true)
 |-- Payment Source Code: string (nullable = true)
 |-- Payment Category Description: string (nullable = true)
 |-- Payment Type Description: string (nullable = true)
 |-- Payment Type Code: string (nullable = true)
 |-- Payment Type ID: integer (nullable = true)
 |-- Debt Type Code: string (nullable = true)
 |-- Debt Type Description: string (nullabl

In [6]:
%%bash
# copy local CSV file to S3 using AWS CLI
# aws s3 cp data/TOP_FedClct_20221001_20221001.csv s3://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv
aws s3 cp data/TOP_FedClct_20221001_20221001.csv s3://daab-dtl-lab-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv
    
#aws s3 ls s3://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv    
aws s3 ls s3://daab-dtl-lab-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv    

upload: data/TOP_FedClct_20221001_20221001.csv to s3://daab-dtl-lab-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv
2022-11-23 20:05:07     920049 TOP_FedClct_20221001_20221001.csv


In [7]:
# Read the CSV file from S3 using Python AWS SDK
import boto3
s3 = boto3.client('s3')

#s3_bucket = 'wc2h-dtl-dev-datawork'
s3_bucket = 'daab-dtl-lab-datawork'
s3_key = 'rmyers07/data/TOP_FedClct_20221001_20221001.csv'

s3_obj = s3.get_object(Bucket=s3_bucket, Key=s3_key )
data = s3_obj['Body'].read().decode('utf-8')

for n,line in enumerate(data.split('\n')):
    print(line)
    if n > 5:
        break


Record Date,Creditor Agency Name,Agency ID,Agency Site Type Code,Agency Site ID,Agency Site Name,Source Agency Alpha,Source Agency Numeric,Source Agency Site ID,Agency Name,Payment Agency ID,Agency Type Code,Payment Source Description,Payment Source Code,Payment Category Description,Payment Type Description,Payment Type Code,Payment Type ID,Debt Type Code,Debt Type Description,Taxable Indicator,Total Gross Amount,Total Net Amount,Total Gross Count,Total Net Count,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number
2022-10-01,Bureau of the Fiscal Service,83,Federal,33MK,33-MK-BUREAU OF FISCAL SERVICE,33,51,MK,Internal Revenue Service,132,Federal,Tax Refund,TAX,TDO,Tax Refund EFT,IE,12,FN,FEDERAL NON-TAX,N,986.00,986.00,1,1,2023,1,2022,4,10,01
2022-10-01,Bureau of the Fiscal Service,83,Federal,33M7,33-M7-FISCAL SERVICE - DMSC,33,1C,M7,Department of Homeland Security,21,State,Vendor payment,VEN,TDO,Vendor EFT,VE,13,FN,FEDERAL N

In [8]:
# Read the CSV file from S3 into a Spark data frame
#s3_url_input = 's3a://wc2h-dtl-dev-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv'
s3_url_input = 's3a://daab-dtl-lab-datawork/rmyers07/data/TOP_FedClct_20221001_20221001.csv'

df = spark.read \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load(s3_url_input) 

df.printSchema()
df.show(5)


22/11/23 20:05:07 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
root
 |-- Record Date: timestamp (nullable = true)
 |-- Creditor Agency Name: string (nullable = true)
 |-- Agency ID: integer (nullable = true)
 |-- Agency Site Type Code: string (nullable = true)
 |-- Agency Site ID: string (nullable = true)
 |-- Agency Site Name: string (nullable = true)
 |-- Source Agency Alpha: string (nullable = true)
 |-- Source Agency Numeric: string (nullable = true)
 |-- Source Agency Site ID: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Payment Agency ID: string (nullable = true)
 |-- Agency Type Code: string (nullable = true)
 |-- Payment Source Description: string (nullable = true)
 |-- Payment Source Code: string (nullable = true)
 |-- Payment Category Description: string (nullable = true)
 |-- Payment Type Description: string (nullable = true)
 |-- Payment Type Code: string (nullable = 

In [9]:
%%bash
# Use AWS CLI to list objects in publicly-accessible S3 parquet data set (AWS US East Region)
aws s3 ls s3://amazon-reviews-pds/parquet/product_category=Electronics/

# WC2H fails , DAAB LAB works 

2018-04-09 06:37:54  100692722 part-00000-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:54  100532983 part-00001-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:54  100519913 part-00002-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:55   99856078 part-00003-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:55  100085478 part-00004-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:57  100470050 part-00005-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:57  100832865 part-00006-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:57  101393971 part-00007-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:58  100623619 part-00008-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet
2018-04-09 06:37:58  100274810 part-00009-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet


In [10]:
# Read a publicly-accessible S3 parquet data set into a Spark dataframe (AWS US East Region)
df = spark.read.parquet("s3a://amazon-reviews-pds/parquet/product_category=Electronics/")
df.printSchema()
df.show(5)

# WC2H fails , DAAB LAB works 

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- year: integer (nullable = true)



+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   51163966|R2RX7KLOQQ5VBG|B00000JBAT|     738692522|Diamond Rio Digit...|          3|            0|          0|   N|                N|Why just 30 minutes?|RIO is really gre...| 1999-06-22|1999|
|         US|   30050581| RPHMRNCGZF2HN|B001BRPLZU|     197287809|NG 283220 AC Adap...|          5|            0|          0|   N|                Y|          Five Stars